In [ ]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip

--2025-05-10 19:15:05--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 183.131.227.248
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|183.131.227.248|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540689175 (516M) [application/zip]
Saving to: ‘train_sample.zip’

train_sample.zip    100%[===================>] 515.64M  3.43MB/s    in 1m 47s  

2025-05-10 19:16:53 (4.81 MB/s) - ‘train_sample.zip’ saved [540689175/540689175]



In [ ]:
!unzip -qq train_sample.zip
!\rm train_sample.zip

In [ ]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip

--2025-05-10 19:19:06--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 183.131.227.248
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|183.131.227.248|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1092637852 (1.0G) [application/zip]
Saving to: ‘test_a.zip’

test_a.zip          100%[===================>]   1.02G  5.46MB/s    in 4m 8s   

2025-05-10 19:23:15 (4.20 MB/s) - ‘test_a.zip’ saved [1092637852/1092637852]



In [ ]:
!unzip -qq test_a.zip
!\rm test_a.zip

In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple pandas

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple scikit-learn

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -i https://mirrors.aliyun.com/pypi/simple resampy

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install librosa -i https://pypi.tuna.tsinghua.edu.cn/simple --default-timeout=100
!pip install resampy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
# 必要的PyTorch库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import torchvision
import torchvision.transforms as transforms


In [ ]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import random

# 特征提取以及数据集的建立

In [ ]:

# 初始化特征和标签列表
feature = []
label = []

# 建立类别标签，不同类别对应不同的数字
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2, 'candied_fruits': 3, 'carrots': 4, 'chips': 5,
              'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream': 11,
              'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon': 17,
              'soup': 18, 'wings': 19}

# 创建一个反向字典，用于从数字映射回类别名称
label_dict_inv = {v: k for k, v in label_dict.items()}

# 定义特征提取函数
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav", augment=False):
    """
    从音频文件中提取特征和标签。

    参数:
    - parent_dir: 数据集的父目录
    - sub_dirs: 子目录列表，每个子目录对应一个类别
    - max_file: 每个类别最多处理的文件数量
    - file_ext: 文件扩展名，默认为 "*.wav"

    返回:
    - feature: 提取的特征列表
    - label: 对应的标签列表
    """
    c = 0  # 初始化计数器
    label, feature = [], []  # 初始化标签和特征列表

    # 遍历每个子目录
    for sub_dir in sub_dirs:
        # 遍历子目录中的音频文件，最多处理 max_file 个文件
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]):
            # 获取文件的类别标签（子目录名称）
            label_name = fn.split('/')[-2]
            # 将类别标签转换为数字，并添加到标签列表中
            label.extend([label_dict[label_name]])
            # 加载音频文件
            X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
            # 数据增强
            if augment:
                X = augment_audio(X, sample_rate)
            # 计算梅尔频谱（mel spectrogram），并取其均值作为特征
            mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            # 将特征添加到特征列表中
            feature.extend([mels])
    
    # 返回特征和标签
    return [feature, label]

# 数据增强函数
def augment_audio(X, sample_rate):
    # 随机添加高斯噪声
    if random.random() < 0.5:
        noise = np.random.randn(len(X))
        X = X + 0.005 * noise
    # 随机时移
    if random.random() < 0.5:
        shift = np.random.randint(sample_rate // 10)
        X = np.roll(X, shift)
    # 随机音量扰动
    if random.random() < 0.5:
        X = X * (0.8 + 0.4 * np.random.rand())
    return X


In [ ]:
# 加载训练数据
# 定义训练数据的父目录
parent_dir = './train_sample/'

# 定义保存目录（当前未使用）
save_dir = "./"

# 定义类别名称数组，每个类别对应一个子目录
folds = sub_dirs = np.array(['aloe', 'burger', 'cabbage', 'candied_fruits',
                             'carrots', 'chips', 'chocolate', 'drinks', 'fries',
                             'grapes', 'gummies', 'ice-cream', 'jelly', 'noodles', 'pickles',
                             'pizza', 'ribs', 'salmon', 'soup', 'wings'])

# 调用特征提取函数，从音频文件中提取特征和标签
# 参数包括父目录、子目录列表以及每个类别最多处理的文件数量
# 数据增强开关
AUGMENT = True
X, Y = extract_features(parent_dir, sub_dirs, max_file=100, augment=AUGMENT)


 76%|███████▌  | 25/33 [00:59<00:18,  2.30s/it]

In [ ]:
# 数据分割，将数据集划分为训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

# 转换为NumPy数组，确保数据格式一致
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# 调整数据形状为 (N, 1, 16, 8)，以适配卷积神经网络的输入格式
X_train = X_train.reshape(-1, 16, 8, 1).transpose(0, 3, 1, 2)
X_test = X_test.reshape(-1, 16, 8, 1).transpose(0, 3, 1, 2)

# 转换为PyTorch张量，创建训练集和测试集的数据集对象
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(Y_train))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(Y_test))


In [ ]:
# 数据加载器
# 定义批量大小
batch_size = 64

# 创建训练数据加载器，支持随机打乱数据
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 创建测试数据加载器，不打乱数据
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
# 定义音频分类器模型
class AudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # 定义卷积层部分
        self.conv_layers = nn.Sequential(
            # 第一个卷积层，输入通道为1，输出通道为64，卷积核大小为3x3，使用padding=1保持输入输出尺寸一致
            nn.Conv2d(1, 64, 3, padding=1),
            # 批归一化层，规范化卷积层输出，加速收敛
            nn.BatchNorm2d(64),
            # 使用Tanh激活函数
            nn.Tanh(),
            # 最大池化层，池化窗口大小为2x2
            nn.MaxPool2d(2),
            # 第二个卷积层，输入通道为64，输出通道为128，卷积核大小为3x3，使用padding=1
            nn.Conv2d(64, 128, 3, padding=1),
            # 批归一化层
            nn.BatchNorm2d(128),
            # 使用Tanh激活函数
            nn.Tanh(),
            # 最大池化层，池化窗口大小为2x2
            nn.MaxPool2d(2),
            # Dropout层，防止过拟合，丢弃率为0.3
            nn.Dropout(0.3)
        )
        # 定义全连接层部分
        self.fc_layers = nn.Sequential(
            # 展平层，将多维张量展平为一维
            nn.Flatten(),
            # 全连接层，输入大小为128 * 4 * 2（经过卷积和池化后的特征图大小），输出大小为1024
            nn.Linear(128 * 4 * 2, 1024),
            # 使用Tanh激活函数
            nn.Tanh(),
            # Dropout层，丢弃率为0.3
            nn.Dropout(0.3),
            # 全连接层，输入大小为1024，输出大小为20（对应20个类别）
            nn.Linear(1024, 20)
        )

    def forward(self, x):
        # 前向传播，先通过卷积层部分
        x = self.conv_layers(x)
        # 再通过全连接层部分
        return self.fc_layers(x)

# DenseNet和ResNet实现
from torch.nn import functional as F

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(nn.Sequential(
                nn.BatchNorm2d(in_channels + i * growth_rate),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels + i * growth_rate, growth_rate, 3, padding=1, bias=False)
            ))
    def forward(self, x):
        features = [x]
        for layer in self.layers:
            out = layer(torch.cat(features, 1))
            features.append(out)
        return torch.cat(features, 1)

class DenseNetSmall(nn.Module):
    def __init__(self, num_classes=20):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.block1 = DenseBlock(32, 16, 3)
        self.trans1 = nn.Conv2d(32+16*3, 64, 1)
        self.pool1 = nn.MaxPool2d(2)
        self.block2 = DenseBlock(64, 16, 3)
        self.trans2 = nn.Conv2d(64+16*3, 128, 1)
        self.pool2 = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128*4*2, num_classes)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.block1(x)
        x = F.relu(self.trans1(x))
        x = self.pool1(x)
        x = self.block2(x)
        x = F.relu(self.trans2(x))
        x = self.pool2(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNetSmall(nn.Module):
    def __init__(self, num_classes=20):
        super().__init__()
        self.conv = nn.Conv2d(1, 32, 3, padding=1)
        self.layer1 = BasicBlock(32, 64, stride=2)
        self.layer2 = BasicBlock(64, 128, stride=2)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128*4*2, num_classes)
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# 原始模型增加正则化
class AudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 2, 1024),
            nn.Tanh(),
            nn.Dropout(0.3),
            nn.Linear(1024, 20)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        return self.fc_layers(x)

In [ ]:
# 选择模型类型：'base', 'densenet', 'resnet'
MODEL_TYPE = 'densenet'  # 可选 'base', 'densenet', 'resnet'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if MODEL_TYPE == 'densenet':
    model = DenseNetSmall(num_classes=20).to(device)
elif MODEL_TYPE == 'resnet':
    model = ResNetSmall(num_classes=20).to(device)
else:
    model = AudioClassifier().to(device)

# L2正则化
optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

[HAMI-core Msg(1781:140142402833280:libvgpu.c:859)]: Initialized


In [ ]:
# 打印模型结构
# 通过打印模型，可以查看模型的各层结构和参数信息
print(model)

AudioClassifier(
  (conv_layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout(p=0.1, inplace=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=1024, bias=True)
    (2): Tanh()
    (3): Linear(in_features=1024, out_features=20, bias=True)
  )
)


In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, 
                num_epochs=100, validate_every=10, show_progress_every=1):
    """
    训练模型并定期进行验证。

    参数:
    - model: 要训练的PyTorch模型。
    - train_loader: 训练数据集的DataLoader。
    - test_loader: 验证数据集的DataLoader。
    - criterion: 损失函数，用于计算模型预测与真实标签之间的误差。
    - optimizer: 优化算法，用于更新模型参数。
    - num_epochs: 训练的总轮数（默认值: 100）。
    - validate_every: 每隔多少轮进行一次验证（默认值: 10）。
    - show_progress_every: 每隔多少轮显示一次进度条（默认值: 1）。
    """
    for epoch in range(num_epochs):
        # 设置模型为训练模式，启用dropout和batch normalization等训练特性
        model.train()
        
        # 初始化训练损失、正确预测计数和样本总数
        train_loss = 0.0  # 累计训练损失
        train_correct = 0  # 累计正确预测的样本数
        train_total = 0  # 累计训练样本总数

        # 控制是否显示进度条
        loader = train_loader if (epoch+1) % show_progress_every == 0 else train_loader
        disable_progress = not ((epoch+1) % show_progress_every == 0)

        # 遍历训练数据集的每个批次
        for inputs, labels in tqdm(loader, 
                                 desc=f"Epoch {epoch+1}/{num_epochs}", 
                                 unit="batch",
                                 disable=disable_progress):
            # 将输入和标签移动到设备（CPU/GPU）
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 清零优化器的梯度，避免梯度累积
            optimizer.zero_grad()
            
            # 前向传播计算输出
            outputs = model(inputs)
            
            # 计算损失，衡量预测与真实标签的差距
            loss = criterion(outputs, labels)
            
            # 反向传播计算梯度
            loss.backward()
            
            # 使用优化器更新模型参数
            optimizer.step()
            
            # 累计训练损失
            train_loss += loss.item()
            
            # 获取预测结果并统计正确预测的数量
            _, predicted = torch.max(outputs.data, 1)  # 获取每个样本的预测类别
            train_total += labels.size(0)  # 累计样本总数
            train_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
        
        # 计算平均训练损失和准确率
        train_loss = train_loss / len(train_loader)  # 平均训练损失
        train_acc = 100 * train_correct / train_total  # 训练准确率（百分比）
        
        # 验证阶段（根据指定频率）
        if (epoch + 1) % validate_every == 0 or epoch == num_epochs - 1:
            # 设置模型为评估模式，禁用dropout等训练特性
            model.eval()
            
            # 初始化验证损失、正确预测计数和样本总数
            val_loss = 0.0  # 累计验证损失
            val_correct = 0  # 累计正确预测的样本数
            val_total = 0  # 累计验证样本总数
            
            # 禁用梯度计算以加速验证过程
            with torch.no_grad():
                # 遍历验证数据集的每个批次
                for inputs, labels in test_loader:
                    # 将输入和标签移动到设备（CPU/GPU）
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    # 前向传播计算输出
                    outputs = model(inputs)
                    
                    # 计算损失
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    
                    # 获取预测结果并统计正确预测的数量
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)  # 累计样本总数
                    val_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
            
            # 计算验证准确率
            val_acc = 100 * val_correct / val_total  # 验证准确率（百分比）
            print(f"\nEpoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
        else:
            # 非验证轮次仅显示训练准确率
            if not disable_progress:
                print(f"\nEpoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.2f}%")
    
    # 训练完成后打印最终结果
    print("\nTraining completed!")
    print(f"Final Train Accuracy: {train_acc:.2f}%")
    
    # 确保最后一个轮次进行验证（如果未在最后一轮验证）
    # 如果最后一个轮次未进行验证，则在训练结束后进行一次最终验证
    if (num_epochs - 1) % validate_every != 0:
        # 设置模型为评估模式，禁用dropout等训练特性
        model.eval()
        
        # 初始化验证损失、正确预测计数和样本总数
        val_loss = 0.0  # 累计验证损失
        val_correct = 0  # 累计正确预测的样本数
        val_total = 0  # 累计验证样本总数
        
        # 禁用梯度计算以加速验证过程
        with torch.no_grad():
            # 遍历验证数据集的每个批次
            for inputs, labels in test_loader:
                # 将输入和标签移动到设备（CPU/GPU）
                inputs, labels = inputs.to(device), labels.to(device)
                
                # 前向传播计算输出
                outputs = model(inputs)
                
                # 计算损失
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # 获取预测结果并统计正确预测的数量
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)  # 累计样本总数
                val_correct += (predicted == labels).sum().item()  # 累计正确预测的样本数
        
        # 计算最终验证准确率
        val_acc = 100 * val_correct / val_total  # 验证准确率（百分比）
        print(f"Final Validation Accuracy: {val_acc:.2f}%")

In [ ]:
# 训练模型
# 调用 train_model 函数，开始训练音频分类器模型
# 参数说明：
# - model: 要训练的模型实例
# - train_loader: 训练数据的 DataLoader
# - test_loader: 测试数据的 DataLoader，用于验证模型性能
# - criterion: 损失函数，用于计算预测误差
# - optimizer: 优化器，用于更新模型参数
# - num_epochs: 训练的总轮数，这里设置为 500
# - validate_every: 每隔 50 个 epoch 进行一次验证
# - show_progress_every: 每隔 50 个 epoch 显示一次训练进度
train_model(model, train_loader, test_loader, criterion, optimizer, 
           num_epochs=500, validate_every=50, show_progress_every=50)


Epoch 55/550: 100%|██████████| 13/13 [00:00<00:00, 247.95batch/s]



Epoch 55/550
Train Loss: 0.0001 | Train Acc: 100.00%
Val Loss: 7.6921 | Val Acc: 38.50%
--------------------------------------------------


Epoch 110/550: 100%|██████████| 13/13 [00:00<00:00, 271.36batch/s]



Epoch 110/550
Train Loss: 0.0000 | Train Acc: 100.00%
Val Loss: 7.6659 | Val Acc: 38.50%
--------------------------------------------------


Epoch 165/550: 100%|██████████| 13/13 [00:00<00:00, 270.13batch/s]


...


Epoch 440/550
Train Loss: 0.0007 | Train Acc: 100.00%
Val Loss: 7.7391 | Val Acc: 39.00%
--------------------------------------------------


Epoch 495/550: 100%|██████████| 13/13 [00:00<00:00, 269.27batch/s]



Epoch 495/550
Train Loss: 0.0000 | Train Acc: 100.00%
Val Loss: 7.8197 | Val Acc: 40.00%
--------------------------------------------------


Epoch 550/550: 100%|██████████| 13/13 [00:00<00:00, 269.12batch/s]


Epoch 550/550
Train Loss: 0.0000 | Train Acc: 100.00%
Val Loss: 7.8078 | Val Acc: 39.50%
--------------------------------------------------

Training completed!
Final Train Accuracy: 100.00%
Final Validation Accuracy: 39.50%


In [ ]:
def extract_test_features(test_dir, max_file=None, file_ext="*.wav"):
    """
    从测试音频文件中提取特征。

    参数:
    - test_dir: 测试数据的目录路径。
    - max_file: 最多处理的文件数量（默认值: None，表示处理所有文件）。
    - file_ext: 文件扩展名，默认为 "*.wav"。

    返回:
    - features: 提取的特征，作为 PyTorch FloatTensor。
    - file_list: 处理的文件路径列表。
    """
    features = []  # 初始化特征列表
    file_list = glob.glob(os.path.join(test_dir, file_ext))  # 获取指定目录下的所有音频文件
    if max_file is not None:  # 如果指定了最大文件数量
        file_list = file_list[:max_file]  # 截取前 max_file 个文件
    
    # 遍历文件列表，提取每个文件的特征
    for fn in tqdm(file_list):
        # 加载音频文件
        X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
        # 计算梅尔频谱（mel spectrogram），并取其均值作为特征
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        features.append(mels)  # 将特征添加到特征列表中
    
    features = np.array(features)  # 转换为 NumPy 数组
    # 返回特征和文件列表
    return torch.FloatTensor(features), file_list

def predict(model, test_features):
    """
    使用模型对测试特征进行预测。

    参数:
    - model: 训练好的 PyTorch 模型。
    - test_features: 测试数据的特征，作为 PyTorch 张量。

    返回:
    - predictions: 预测的类别名称列表。
    """
    model.eval()  # 设置模型为评估模式，禁用 dropout 等训练特性
    with torch.no_grad():  # 禁用梯度计算以加速预测过程
        test_features = test_features.to(device)  # 将特征移动到设备（CPU/GPU）
        outputs = model(test_features)  # 前向传播计算输出
        _, predicted = torch.max(outputs.data, 1)  # 获取每个样本的预测类别
        # 将预测的类别数字映射回类别名称
        return [label_dict_inv[p.item()] for p in predicted]



In [ ]:
test_features, file_list = extract_test_features('./test_a/')
# 调整测试特征的形状以适配模型输入
# 这里将特征调整为 (N, 1, 16, 8) 的形状，其中 N 是样本数量

100%|██████████| 2000/2000 [1:04:20<00:00,  1.93s/it]


In [ ]:
# 调整测试特征的形状为 (N, 1, 16, 8)，其中 N 是样本数量
# 1 表示单通道（灰度图像），16 和 8 是特征的高和宽
# 这种形状适配卷积神经网络的输入格式
test_features = test_features.view(-1, 1, 16, 8)


In [ ]:
# 使用训练好的模型对测试特征进行预测
# 参数:
# - model: 训练好的音频分类器模型
# - test_features: 提取并调整形状后的测试数据特征
# 返回:
# - predictions: 测试数据的预测类别名称列表
predictions = predict(model, test_features)


In [ ]:
# 将预测结果赋值给变量 preds
preds = predictions

# 获取测试文件的路径列表
path = glob.glob('./test_a/*.wav')

# 创建一个 DataFrame，将文件路径和预测标签对应起来
result = pd.DataFrame({'name': path, 'label': preds})

# 提取文件名（去掉路径部分），并更新到 'name' 列
result['name'] = result['name'].apply(lambda x: x.split('/')[-1])

# 将结果保存为 CSV 文件，文件名为 'submit.csv'，不包含索引列
result.to_csv('submit.csv', index=None)

In [ ]:
# 统计测试数据目录中以 .wav 为扩展名的音频文件数量
!ls ./test_a/*.wav | wc -l

2000


In [ ]:
# 使用 wc -l 命令统计 submit.csv 文件中的行数
# 这可以用来检查提交文件中包含的记录数量
!wc -l submit.csv

2001 submit.csv
